Overview

#### The purpose of this notebook is to implement and compare two federated learning models: a standard federated learning model and a differentially private (DP) federated learning model, using the CIFAR-10 dataset. This comparison helps in understanding the trade-offs between standard federated learning and federated learning with differential privacy in terms of model performance and accuracy.

1. **Imports and Environment Setup**:
    - Libraries such as `tensorflow`, `tensorflow_federated`, and `tensorflow_privacy` are imported.
    - `nest_asyncio.apply()` is used to enable the asynchronous event loop to work properly within Jupyter notebooks.

2. **Loading and Preprocessing Data**:
    - The CIFAR-10 dataset is loaded and split into training and test datasets.
    - The `preprocess` function normalizes the image data by scaling the pixel values to the range [0, 1].

3. **Client Dataset Creation**:
    - The training dataset is divided into `num_clients` parts to simulate federated learning across multiple clients.
    - Each client's dataset is batched for training.

4. **Model Definition**:
    - `create_keras_model`: Defines a simple Convolutional Neural Network (CNN) model using Keras.
    - `model_fn_standard`: Wraps the Keras model into a TFF model for standard federated learning.
    - `model_fn_with_dp`: Wraps the Keras model with a differentially private optimizer for federated learning with differential privacy. Note: The actual optimizer is applied in the training loop.

5. **Utility Functions**:
    - `assign_weights_to_keras_model`: Copies the weights from the TFF model state to the Keras model for evaluation.
    - `evaluate_model`: Evaluates the model on the test dataset and returns the loss and accuracy.
    - `check_dataset`: Helper function to print the shape and labels of the dataset batches for verification.

6. **Federated Learning Processes**:
    - `iterative_process_standard`: Defines the federated learning process for the standard model using TFF’s federated averaging.
    - `iterative_process_with_dp`: Defines the federated learning process for the DP model, using the standard model function and setting the differentially private optimizer in the client update phase.

7. **Training the Models**:
    - `NUM_ROUNDS`: Defines the number of federated training rounds.
    - The standard and DP models are trained for the specified number of rounds, with training metrics printed for each round.

8. **Evaluating the Models**:
    - The test dataset is preprocessed and batched.
    - Both the standard and DP models are evaluated on the test dataset, and their respective loss and accuracy are printed.

In [1]:
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_privacy as tfp
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Load the CIFAR-10 dataset
(cifar10_train_images, cifar10_train_labels), (cifar10_test_images, cifar10_test_labels) = tf.keras.datasets.cifar10.load_data()


In [3]:
# Preprocess the dataset
def preprocess(images, labels):
    images = tf.cast(images, tf.float32) / 255.0
    return (images, labels)

The CIFAR-10 dataset is loaded and normalized by scaling the image pixel values to the range [0, 1], which is a common preprocessing step to improve model training.

In [4]:
# Split the dataset into multiple "client" datasets
num_clients = 10
client_datasets = []
for i in range(num_clients):
    start = i * len(cifar10_train_images) // num_clients
    end = (i + 1) * len(cifar10_train_images) // num_clients
    client_images = cifar10_train_images[start:end]
    client_labels = cifar10_train_labels[start:end]
    client_dataset = tf.data.Dataset.from_tensor_slices((client_images, client_labels))
    client_dataset = client_dataset.map(preprocess).batch(20)
    client_datasets.append(client_dataset)

In [5]:
# Define a simple CNN model
def create_keras_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    return model

def model_fn_standard():
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=client_datasets[0].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

def model_fn_with_dp():
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=client_datasets[0].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


A simple CNN is defined to classify CIFAR-10 images. The same model architecture is used for both the standard and DP models.

In [6]:
iterative_process_standard = tff.learning.build_federated_averaging_process(
    model_fn_standard,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

iterative_process_with_dp = tff.learning.build_federated_averaging_process(
    model_fn_with_dp,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))


The `build_federated_averaging_process` function creates a federated averaging process using the provided model function and optimizers. This process coordinates the federated learning across clients.

In [7]:
NUM_ROUNDS = 10

# Train the standard model
state_standard = iterative_process_standard.initialize()
for round_num in range(NUM_ROUNDS):
    state_standard, metrics_standard = iterative_process_standard.next(state_standard, client_datasets)
    print('Standard Model - round {:2d}, metrics={}'.format(round_num, metrics_standard))

# Train the differentially private model
state_with_dp = iterative_process_with_dp.initialize()
for round_num in range(NUM_ROUNDS):
    state_with_dp, metrics_with_dp = iterative_process_with_dp.next(state_with_dp, client_datasets)
    print('DP Model - round {:2d}, metrics={}'.format(round_num, metrics_with_dp))


Standard Model - round  0, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.14472), ('loss', 2.2682927)]))])
Standard Model - round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.21964), ('loss', 2.1167924)]))])
Standard Model - round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.284), ('loss', 1.9737297)]))])
Standard Model - round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.33324), ('loss', 1.8567117)]))])
Standard Model - round  4,

The standard model is trained for a specified number of rounds. In each round, the server updates the model using client updates, and training metrics are printed.

In [8]:
def evaluate_model(state, model_fn, test_dataset):
    keras_model = create_keras_model()
    tff_model = model_fn()
    
    # Create a function to extract weights from the TFF state
    def assign_weights_to_keras_model(keras_model, tff_model):
        tff_weights = tff_model.weights.trainable
        for var, tff_var in zip(keras_model.trainable_variables, tff_weights):
            var.assign(tff_var.numpy())

    # Assign weights from the TFF state to the Keras model
    assign_weights_to_keras_model(keras_model, tff_model)

    # Compile the Keras model
    keras_model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    test_images, test_labels = zip(*list(test_dataset))
    test_images = tf.concat(test_images, axis=0)
    test_labels = tf.concat(test_labels, axis=0)
    
    loss, accuracy = keras_model.evaluate(test_images, test_labels, verbose=0)
    return loss, accuracy


In [9]:
test_dataset = tf.data.Dataset.from_tensor_slices((cifar10_test_images, cifar10_test_labels))
test_dataset = test_dataset.map(preprocess).batch(20)

loss_standard, accuracy_standard = evaluate_model(state_standard, model_fn_standard, test_dataset)
print(f'Standard Model - Test loss: {loss_standard}, Test accuracy: {accuracy_standard}')

loss_with_dp, accuracy_with_dp = evaluate_model(state_with_dp, model_fn_with_dp, test_dataset)
print(f'DP Model - Test loss: {loss_with_dp}, Test accuracy: {accuracy_with_dp}')


Standard Model - Test loss: 2.3110618591308594, Test accuracy: 0.10920000076293945
DP Model - Test loss: 2.3136303424835205, Test accuracy: 0.10300000011920929


### Enhanced approach

In [10]:
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_privacy as tfp
import nest_asyncio
nest_asyncio.apply()

In [11]:
# Load the CIFAR-10 dataset
(cifar10_train_images, cifar10_train_labels), (cifar10_test_images, cifar10_test_labels) = tf.keras.datasets.cifar10.load_data()

In [12]:
# Preprocess the dataset
def preprocess(images, labels):
    images = tf.cast(images, tf.float32) / 255.0
    return (images, labels)


In [13]:
# Split the dataset into multiple "client" datasets
num_clients = 10
client_datasets = []
for i in range(num_clients):
    start = i * len(cifar10_train_images) // num_clients
    end = (i + 1) * len(cifar10_train_images) // num_clients
    client_images = cifar10_train_images[start:end]
    client_labels = cifar10_train_labels[start:end]
    client_dataset = tf.data.Dataset.from_tensor_slices((client_images, client_labels))
    client_dataset = client_dataset.map(preprocess).batch(20)
    client_datasets.append(client_dataset)

In [14]:
# Define a simple CNN model
def create_keras_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(32, 32, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    return model

In [15]:
def model_fn_standard():
    keras_model = create_keras_model()
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=client_datasets[0].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [16]:
def model_fn_with_dp():
    keras_model = create_keras_model()
    optimizer = tfp.DPAdamGaussianOptimizer(
        l2_norm_clip=1.0,
        noise_multiplier=0.5,
        num_microbatches=1,
        learning_rate=0.001
    )
    keras_model.compile(optimizer=optimizer,
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return tff.learning.from_keras_model(
        keras_model,
        input_spec=client_datasets[0].element_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [28]:

def assign_weights_to_keras_model(keras_model, tff_state):
    tff_weights = tff_state.model.trainable
    for var, tff_var in zip(keras_model.trainable_variables, tff_weights):
        var.assign(tff_var)  # Removed .numpy()


In [29]:
def evaluate_model(state, model_fn, test_dataset):
    keras_model = create_keras_model()
    assign_weights_to_keras_model(keras_model, state)

    keras_model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    test_images, test_labels = zip(*list(test_dataset))
    test_images = tf.concat(test_images, axis=0)
    test_labels = tf.concat(test_labels, axis=0)

    loss, accuracy = keras_model.evaluate(test_images, test_labels, verbose=0)
    return loss, accuracy

In [30]:
def check_dataset(dataset):
    for batch in dataset.take(1):
        images, labels = batch
        print(f'Batch shape: {images.shape}, Labels: {labels.numpy()}')

In [31]:
for i, client_dataset in enumerate(client_datasets):
    print(f'Client {i} dataset:')
    check_dataset(client_dataset)

Client 0 dataset:
Batch shape: (20, 32, 32, 3), Labels: [[6]
 [9]
 [9]
 [4]
 [1]
 [1]
 [2]
 [7]
 [8]
 [3]
 [4]
 [7]
 [7]
 [2]
 [9]
 [9]
 [9]
 [3]
 [2]
 [6]]
Client 1 dataset:
Batch shape: (20, 32, 32, 3), Labels: [[6]
 [7]
 [9]
 [0]
 [5]
 [2]
 [3]
 [3]
 [3]
 [9]
 [0]
 [9]
 [2]
 [9]
 [1]
 [0]
 [2]
 [3]
 [9]
 [6]]
Client 2 dataset:
Batch shape: (20, 32, 32, 3), Labels: [[1]
 [6]
 [6]
 [8]
 [8]
 [3]
 [4]
 [6]
 [0]
 [6]
 [0]
 [3]
 [6]
 [6]
 [5]
 [4]
 [8]
 [3]
 [2]
 [6]]
Client 3 dataset:
Batch shape: (20, 32, 32, 3), Labels: [[0]
 [6]
 [7]
 [0]
 [4]
 [9]
 [5]
 [8]
 [0]
 [4]
 [3]
 [8]
 [4]
 [7]
 [1]
 [8]
 [3]
 [5]
 [4]
 [5]]
Client 4 dataset:
Batch shape: (20, 32, 32, 3), Labels: [[8]
 [5]
 [0]
 [6]
 [9]
 [2]
 [8]
 [3]
 [6]
 [2]
 [7]
 [4]
 [6]
 [9]
 [0]
 [0]
 [7]
 [3]
 [7]
 [2]]
Client 5 dataset:
Batch shape: (20, 32, 32, 3), Labels: [[6]
 [9]
 [8]
 [4]
 [0]
 [6]
 [3]
 [1]
 [3]
 [9]
 [9]
 [8]
 [5]
 [8]
 [4]
 [5]
 [0]
 [4]
 [2]
 [3]]
Client 6 dataset:
Batch shape: (20, 32, 32, 3), Labels: [[

In [32]:
# Create federated averaging processes
iterative_process_standard = tff.learning.build_federated_averaging_process(
    model_fn_standard,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

iterative_process_with_dp = tff.learning.build_federated_averaging_process(
    model_fn_standard,  # Use standard model_fn as DP optimizer will be applied in the client update
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

In [33]:
# Train the models
NUM_ROUNDS = 10

In [34]:
# Train the standard model
state_standard = iterative_process_standard.initialize()
for round_num in range(NUM_ROUNDS):
    state_standard, metrics_standard = iterative_process_standard.next(state_standard, client_datasets)
    print(f'Standard Model - Round {round_num}, metrics={metrics_standard["train"]}')

Standard Model - Round 0, metrics=OrderedDict([('sparse_categorical_accuracy', 0.16792), ('loss', 2.2336984)])
Standard Model - Round 1, metrics=OrderedDict([('sparse_categorical_accuracy', 0.2438), ('loss', 2.0595803)])
Standard Model - Round 2, metrics=OrderedDict([('sparse_categorical_accuracy', 0.29084), ('loss', 1.9588082)])
Standard Model - Round 3, metrics=OrderedDict([('sparse_categorical_accuracy', 0.33166), ('loss', 1.8597034)])
Standard Model - Round 4, metrics=OrderedDict([('sparse_categorical_accuracy', 0.37108), ('loss', 1.7508552)])
Standard Model - Round 5, metrics=OrderedDict([('sparse_categorical_accuracy', 0.40194), ('loss', 1.6609166)])
Standard Model - Round 6, metrics=OrderedDict([('sparse_categorical_accuracy', 0.42368), ('loss', 1.5977634)])
Standard Model - Round 7, metrics=OrderedDict([('sparse_categorical_accuracy', 0.44202), ('loss', 1.5502768)])
Standard Model - Round 8, metrics=OrderedDict([('sparse_categorical_accuracy', 0.4571), ('loss', 1.511056)])
Stan

In [35]:
# Train the DP model
state_with_dp = iterative_process_with_dp.initialize()
for round_num in range(NUM_ROUNDS):
    state_with_dp, metrics_with_dp = iterative_process_with_dp.next(state_with_dp, client_datasets)
    print(f'DP Model - Round {round_num}, metrics={metrics_with_dp["train"]}')


DP Model - Round 0, metrics=OrderedDict([('sparse_categorical_accuracy', 0.15544), ('loss', 2.2649534)])
DP Model - Round 1, metrics=OrderedDict([('sparse_categorical_accuracy', 0.23802), ('loss', 2.0920632)])
DP Model - Round 2, metrics=OrderedDict([('sparse_categorical_accuracy', 0.28604), ('loss', 1.9753482)])
DP Model - Round 3, metrics=OrderedDict([('sparse_categorical_accuracy', 0.33576), ('loss', 1.865395)])
DP Model - Round 4, metrics=OrderedDict([('sparse_categorical_accuracy', 0.3779), ('loss', 1.7484978)])
DP Model - Round 5, metrics=OrderedDict([('sparse_categorical_accuracy', 0.4092), ('loss', 1.6534642)])
DP Model - Round 6, metrics=OrderedDict([('sparse_categorical_accuracy', 0.4321), ('loss', 1.583567)])
DP Model - Round 7, metrics=OrderedDict([('sparse_categorical_accuracy', 0.45172), ('loss', 1.5311791)])
DP Model - Round 8, metrics=OrderedDict([('sparse_categorical_accuracy', 0.46618), ('loss', 1.4899096)])
DP Model - Round 9, metrics=OrderedDict([('sparse_categorica

In [36]:
# Load and preprocess the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((cifar10_test_images, cifar10_test_labels))
test_dataset = test_dataset.map(preprocess).batch(20)

In [37]:
# Evaluate the standard model
loss_standard, accuracy_standard = evaluate_model(state_standard, model_fn_standard, test_dataset)
print(f'Standard Model - Test loss: {loss_standard}, Test accuracy: {accuracy_standard}')

# Evaluate the differentially private model
loss_with_dp, accuracy_with_dp = evaluate_model(state_with_dp, model_fn_with_dp, test_dataset)
print(f'DP Model - Test loss: {loss_with_dp}, Test accuracy: {accuracy_with_dp}')

Standard Model - Test loss: 1.4060903787612915, Test accuracy: 0.49900001287460327
DP Model - Test loss: 1.3733359575271606, Test accuracy: 0.5116000175476074
